In [1]:
from utils import *
from collections import defaultdict
import itertools

In [2]:
data = dl_input(18)

In [32]:
data

'9 * 7 + ((6 + 9 * 2 + 6 + 7 + 5) * (5 + 9 + 7) * 6) * (4 + 8) + 5 * 8\n3 * (4 * 9 + 8 + 4 * 4 + 7) + 7 * (8 * 9)\n4 * ((6 * 5 * 9) + 2) + 9 * 4\n((4 * 6 * 2 * 9) + 7 * 4) + 4 * 6 + 5 * 6 + 6\n9 * 8 * ((5 + 7 * 8 * 2) * 6) + 3\n6 + (3 * 7) * 3 + (9 * (6 * 6 + 5 + 9 + 9) * 7 * 9 + (4 * 8) + 8)\n6 * 4 + 2 + 8 + 7\n(5 * (9 * 8) * 6) + 2 + 8 * 5 * 4 * 3\n(3 + 2 * 2 * 8 * 5) + (7 + 6 * 4 * 8) * (6 * 9 * 5 * 9)\n(4 * 2 + (9 + 2 * 4) * 5 + 9) + (9 * 3)\n7 * 6 + 6\n4 + (8 + (5 * 9) + 9 * 8)\n9 + 8 + 5 + 4 * 2 * 9\n8 * (5 + (8 + 2) * 9 + 4 + 7) + 8\n2 * 2 + 9 * 6 + (2 * (8 * 2 + 3 * 8) * (9 + 7 * 4 * 8 * 2)) * 2\n4 + (5 * 6 + 7 * 5) + 4\n4 + (9 * 4) + 4 * (4 + 7 + 9 * 4) * 3\n(4 * (2 + 2) * 9 * 8 * 3) + 9 * 3\n6 * (5 * 3 + 4 + (4 + 4 + 5 * 7 * 4)) * 4\n8 * (6 * 9 + 3 * (6 * 6 * 2) + 4 + 9) * (4 * 8 * (9 * 4 + 8 + 7 * 7) * 5) + 4 + 7\n(7 * 3 * 2) + 3 + 8 * (5 + 7 * 3 + 4 * 2 * (8 + 6 * 4)) * 3 * (8 + 9 + 7 * 3 * 3)\n8 * 7 + 4 + 7 * (3 * 3 + 7 + (3 * 5 * 8 + 8 + 3 * 4))\n(7 * (4 * 3 + 8 * 7 * 3 +

In [259]:
l = '((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2'

def parse(t, *l):
    if t == '(': t,l = parse(*l)
        
    while len(l):
        op, *l = l
        if op == ')': return (t, l)
        
        val, *l = l
        if val == '(': val, l = parse(*l)
        if op == '*': t *= val
        if op == '+': t += val
    return t


data = [[c if c in '()*+' else int(c) for c in l.replace('(', '( ').replace(')', ' )').split()] for l in open('18/input.txt')]

sum(parse(*l) for l in data)

510009915468

In [8]:
import ast as a
NT = a.NodeTransformer
cl = a.copy_location

class T1(NT):
    def visit_Sub(_,n): return cl(a.Mult(),n)


def 𝓔(s,T,D):
    s = s.translate(s.maketrans(D))
    return eval(compile(T().visit(a.parse(s,'','eval')),'','eval'))

print('Part 1:',sum(𝓔(l, T1, {'*':'-'}) for l in open('18/input.txt')))

class T2(NT):
    def visit_Add(_,n): return cl(a.Mult(),n)
    def visit_Mult(_,n): return cl(a.Add(),n)
    
print('Part 2:',sum(𝓔(l, T2, {'+':'*','*':'+'}) for l in open('18/input.txt')))

Part 1: 510009915468
Part 2: 321176691637769


In [9]:
import ast as a
NT = a.NodeTransformer
cl = a.copy_location

class T1(NT):
    def visit_Sub(_,n): return cl(a.Mult(),n)

𝓔 = lambda s,T,D: eval(compile(T().visit(a.parse(s.translate(D),'','eval')),'','eval'))

print('Part 1:',sum(𝓔(l, T1, {42:'-'}) for l in open('18/input.txt')))

class T2(NT):
    def visit_Add(_,n): return cl(a.Mult(),n)
    def visit_Mult(_,n): return cl(a.Add(),n)
    
print('Part 2:',sum(𝓔(l, T2, {43:'*',42:'+'}) for l in open('18/input.txt')))

Part 1: 510009915468
Part 2: 321176691637769


In [2]:
import ast as a
NT= a.NodeTransformer

E= lambda e,D,*m: eval(compile(NT().visit(a.parse(e.translate(D),*m)),*m))

def A(o,p): setattr(NT,'visit_'+o,lambda _,n:a.copy_location(p,n))

for D in ({42:'-'},{43:'*',42:'+'}):
    A('Sub', a.Mult())
    print(sum(E(l,D,'','eval') for l in open('18/input.txt')))
    A('Add', a.Mult())
    A('Mult', a.Add())

510009915468
321176691637769


In [13]:
'1 + 2'.translate({'+': '*'})

'1 + 2'

In [230]:
def split_string(s, bys):
    par = 0
    
    print(s)
    
    for by in bys:
        for i,c in enumerate(s):
            if c == '(': par += 1
            if c == ')': par -= 1
            if c in by and par == 0:
                return((c, split_string(s[:i], bys), split_string(s[i+1:], bys)))
            
    if s[0] == '(':
        return split_string(s[1:-1], bys)
    else:
        return s

def eval_tree_infix(tree):
    if not isinstance(tree, tuple):
        return int(tree)
    op, left, right = tree
    left = eval_tree(left)
    right = eval_tree(right)
    if op == '+':
        return left + right
    elif op == '*':
        return left * right

def eval_tree_prefix(tree):
    if not isinstance(tree, tuple):
        return int(tree)
    op, left, right = tree
    if not isinstance(left, tuple):
        return int(left)
    
    left = eval_tree_prefix(left)
    right = eval_tree_prefix(right)
    if op == '+':
        return left + right
    elif op == '*':
        return left * right
    
exps = [l.strip().replace(' ', '') for l in open('18/input.txt')]

# print('Part 1:', sum(eval_tree(split_string(e, ['*+'])) for e in exps))
# print('Part 2:', sum(eval_tree(split_string(e, ['*','+'])) for e in exps))

# print(eval_tree(split_string('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2'.replace(' ', ''), '*+')))
tree = split_string('2 * 3 + (4 * 5)'.replace(' ', ''), '*+')
print(tree)
print(eval_tree_prefix(tree))


# 510009915468
# 321176691637769

2*3+(4*5)
2
3+(4*5)
3
(4*5)
4*5
4
5
('*', '2', ('+', '3', ('*', '4', '5')))
2


In [260]:
parse = lambda line: eval(f"(({ line.replace('+',')+(').replace('*','))*((') }))")

print('Part 2:', sum(parse(e) for e in open('18/input.txt')))

Part 2: 321176691637769


In [67]:
isinstance(tree, tuple)

True